In [1]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [2]:
import addict
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from tqdm.notebook import tqdm

import celeri
celeri = reload(celeri)

plt.rcParams['text.usetex'] = False # Plotting the global model is much much faster with tex fonts turned off

# Read in data files and do basic processing

In [3]:
command_file_name = "./data/western_north_america/basic_command.json"
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure, block = celeri.assign_block_labels(segment, station, block, mogi, sar)
# celeri.plot_block_labels(segment, block, station, closure)

# Create storage dictionaries and calculate elastic operators

In [16]:
assembly = addict.Dict()
operators = addict.Dict()
assembly = celeri.merge_geodetic_data(assembly, station, sar) # Not sure this work correctly

# Elastic operators
operators.okada_segment_station = celeri.get_segment_station_operator_okada(segment, station, command)
# celeri.plot_segment_displacements(segment, station, command, segment_idx=0, strike_slip=1, dip_slip=0, tensile_slip=0, lon_min=235, lon_max=255, lat_min=30, lat_max=50, quiver_scale=1e-1)
operators.tri_station = celeri.get_tri_station_operator_okada(meshes, station, command)
celeri.get_all_mesh_smoothing_matrices(meshes)
celeri.get_all_mesh_smoothing_matrices_simple(meshes, 2)

Calculating Okada partials for segments:   0%|          | 0/837 [00:00<?, ?it/s]

Calculating cutde partials for triangles:   0%|          | 0/1841 [00:00<?, ?it/s]

# Calculate additional non-elastic operators

In [5]:
# Build all linear kinematic operators
operators.block_rotation = celeri.get_block_rotation_operator(station)
operators.global_float_block_rotation = celeri.get_global_float_block_rotation_operator(station)

# TODO: Update celeri.block_constraints to properly use block.
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)
operators.slip_rate_segment_block = celeri.get_fault_slip_rate_partials(segment, block)
operators.block_strain_rate, strain_rate_block_idx = celeri.get_strain_rate_centroid_operator(block, station, segment)
operators.mogi_station = celeri.get_mogi_operator(mogi, station, command)

# Get additional matrix shape information for assembly
assembly.index.sz_elastic = operators.okada_segment_station.shape # Not sure this is correct
assembly.index.sz_slip = operators.slip_rate_segment_block.shape # Not sure this is correct
assembly.index.sz_rotation = operators.block_rotation.shape # Not sure this is correct
assembly = celeri.station_row_keep(assembly) # Not sure this is correct

In [6]:

# Shrink operators.tri_station so that there are no vertical displacments and no tensile slip
tde_matrix = copy.deepcopy(operators.tri_station)
tde_matrix = np.delete(tde_matrix, np.arange(2, tde_matrix.shape[0], 3), axis=0)
tde_matrix = np.delete(tde_matrix, np.arange(2, tde_matrix.shape[1], 3), axis=1)

# Slip estimation with direct inverse and smoothing matrix
smoothing_matrix = 1e2 * meshes[0].smoothing_matrix.toarray()
smoothing_matrix = np.delete(smoothing_matrix, np.arange(2, smoothing_matrix.shape[0], 3), axis=0)
smoothing_matrix = np.delete(smoothing_matrix, np.arange(2, smoothing_matrix.shape[1], 3), axis=1)
smoothing_matrix = meshes[0].smoothing_weight * smoothing_matrix # Weight smoothing matrix
tde_and_smoothing_matrix = np.vstack((tde_matrix, smoothing_matrix))

print(tde_matrix.shape)
print(smoothing_matrix.shape)
print(tde_and_smoothing_matrix.shape)

(3372, 3682)
(3682, 3682)
(7054, 3682)


# Sketching out the assembly of the block model system
The list here is in as of right now (09/19/21)...a work in progress

## The observation vector:    
| What | Symbolically | Variable name |
| - | - | - |
| GPS velocities | $\mathbf{v}$ | ```assembly.data.east_vel, assembly.data.north_vel``` |
| Slip rate constraints | $\dot{\mathbf{s}}$ | ```assembly.slip_rate_constraints``` |
| Rotation vector constraints | $\boldsymbol{\omega}$ | ```assembly.block_constraints``` |

## The state vector:    
| What | Symbolically | Variable name |
| - | - | - |
| Block rotation rates | $\boldsymbol{\Omega}$ | ```estimation.block_rotation_vectors``` |
| TDE slip | $\mathbf{t}$ | ```estimation.tri_slip_rates``` |
  
## Matrix operators:    
| What | Symbolically | Variable name |
| - | - | - |
| Rotation vector to GPS | $\boldsymbol{\Omega} \rightarrow \mathbf{v}$ | ```operators.block_rotation``` | 
| Rotation vector to GPS | $\boldsymbol{\Omega} \rightarrow \mathbf{v}$ | ```operators.global_float_block_rotation``` | 
| Rotation vector to segment slip rates| $\boldsymbol{\Omega} \rightarrow \dot{\mathbf{s}}$ | ```operators.slip_rate_segment_block``` | 
| Segment slip rates to elastic deformation | $\dot{\mathbf{s}} \rightarrow \mathbf{v}$ | ```operators.okada_segment_station``` | 
| TDE slip to GPS | $\mathbf{t} \rightarrow \mathbf{v}$ | ```operators.tri_station``` |
| Rotation vector to slip rate constraints| $\boldsymbol{\Omega} \rightarrow \dot{\mathbf{s}}$ | ```operators.slip_rate_constraints``` |
| Rotation vector to Euler pole contraints | $\mathbf{I}$ | ```operators.block_motion_contraints``` |



# Inversion with block motion only and no elastic deformation
-  Runs and solution looks reasonable

In [7]:
# Rotations only - with JDF a priori and no global float
data_vector = np.zeros(2 * assembly.data.n_stations
                     + 3 * assembly.data.n_block_constraints)
data_vector[0:2 * assembly.data.n_stations] = celeri.interleave2(assembly.data.east_vel, assembly.data.north_vel)

# Add block motion costraints to data vector
data_vector[2 * assembly.data.n_stations:2 * assembly.data.n_stations + 3 * assembly.data.n_block_constraints] = assembly.data.block_constraints
data_vector[2 * assembly.data.n_stations:2 * assembly.data.n_stations + 3 * assembly.data.n_block_constraints] = 0
operator = np.zeros((2 * assembly.data.n_stations + 3 * assembly.data.n_block_constraints, 3 * len(block)))
operator[0:2 * assembly.data.n_stations, :] = np.delete(operators.block_rotation, np.arange(2, operators.block_rotation.shape[0], 3), axis=0) # Delete up velocity partials
operator[2 * assembly.data.n_stations:2 * assembly.data.n_stations + 3 * assembly.data.n_block_constraints, :]  = operators.block_motion_constraints

estimation = addict.Dict()
state_vector = np.linalg.inv(operator.T @ operator) @ operator.T @ data_vector 
estimation.predictions = operator @ state_vector
vel = estimation.predictions[0:2 * assembly.data.n_stations]
estimation.east_vel = vel[0::2]
estimation.north_vel = vel[1::2]
east_vel_rotation_only = vel[0::2]
north_vel_rotation_only = vel[1::2]

# Plot observed and estimated velocities
lon_min=235
lon_max=255
lat_min=30
lat_max=50
quiver_scale=1e2
plt.figure()
for i in range(len(segment)):
    plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-k", linewidth=0.5)
plt.quiver(station.lon, station.lat, station.east_vel, station.north_vel, scale=quiver_scale, scale_units="inches", color="g")
plt.quiver(station.lon, station.lat, estimation.east_vel, estimation.north_vel, scale=quiver_scale, scale_units="inches", color="r")
plt.xlim([lon_min, lon_max])
plt.ylim([lat_min, lat_max])
plt.gca().set_aspect("equal", adjustable="box")
plt.title("observed and estimated velocities - block rotations only")
plt.show()

# Calculate mean squared residual velocity
east_vel_residual = estimation.east_vel - station.east_vel
north_vel_residual = estimation.north_vel - station.north_vel
residual = np.concatenate((east_vel_residual, north_vel_residual))
print(np.sum(residual**2) / len(station))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

12.59569094598103


# Inversion with block rotations and fully locked rectangular segments
- Runs and looks reasonable

In [8]:
# Elastic slip deficit contribution to velocities with fully locked fault segments
slip_rate_to_okada_to_velocities = copy.deepcopy(operators.okada_segment_station)
rotation_to_slip_rate = copy.deepcopy(operators.slip_rate_segment_block)

# Delete rows associated with vertical velocities
slip_rate_to_okada_to_velocities = np.delete(slip_rate_to_okada_to_velocities, np.arange(2, slip_rate_to_okada_to_velocities.shape[0], 3), axis=0)

rotation_to_slip_rate_to_okada_to_velocities = slip_rate_to_okada_to_velocities @ rotation_to_slip_rate
operator = np.zeros((2 * assembly.data.n_stations + 3 * assembly.data.n_block_constraints, 3 * len(block)))
operator[0:2 * assembly.data.n_stations, :] = np.delete(operators.block_rotation, np.arange(2, operators.block_rotation.shape[0], 3), axis=0) - rotation_to_slip_rate_to_okada_to_velocities # Delete up velocity partials
operator[2 * assembly.data.n_stations:2 * assembly.data.n_stations + 3 * assembly.data.n_block_constraints, :] = operators.block_motion_constraints

estimation2 = addict.Dict()
state_vector2 = np.linalg.inv(operator.T @ operator) @ operator.T @ data_vector 
estimation2.predictions = operator @ state_vector2
vel = estimation2.predictions[0:2 * assembly.data.n_stations]
estimation2.east_vel = vel[0::2]
estimation2.north_vel = vel[1::2]

# Plot observed and estimated velocities
lon_min=235
lon_max=255
lat_min=30
lat_max=50
quiver_scale=1e2
plt.figure()
for i in range(len(segment)):
    plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-k", linewidth=0.5)
plt.quiver(station.lon, station.lat, station.east_vel, station.north_vel, scale=quiver_scale, scale_units="inches", color="g")
plt.quiver(station.lon, station.lat, estimation.east_vel, estimation.north_vel, scale=quiver_scale, scale_units="inches", color="r")
plt.xlim([lon_min, lon_max])
plt.ylim([lat_min, lat_max])
plt.gca().set_aspect("equal", adjustable="box")
plt.title("observed and estimated velocities - block rotations and elastic segments")
plt.show()

# Calculate mean squared residual velocity
east_vel_residual = estimation2.east_vel - station.east_vel
north_vel_residual = estimation2.north_vel - station.north_vel
residual = np.concatenate((east_vel_residual, north_vel_residual))
print(np.sum(residual**2) / len(station))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

7.396845959994106


# Plot elastic velocities only

In [9]:
# Calculate elastic velocities
elastic_velocities = rotation_to_slip_rate_to_okada_to_velocities @ state_vector

# Plot elastic velocities
lon_min=235
lon_max=255
lat_min=30
lat_max=50
quiver_scale=1e2
plt.figure()
for i in range(len(segment)):
    plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-k", linewidth=0.5)
plt.quiver(station.lon, station.lat, elastic_velocities[0::2], elastic_velocities[1::2], scale=quiver_scale, scale_units="inches", color="r")
plt.xlim([lon_min, lon_max])
plt.ylim([lat_min, lat_max])
plt.gca().set_aspect("equal", adjustable="box")
plt.title("elastic velocity only -- Looks pretty good!!!")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Block model with block rotations, fully locked segments and partially locked subduction zone using the full tde_matrix and smoothing matrix

In [10]:
# TODO: What should this converstion be?
DEG_PER_MYR_TO_RAD_PER_YR = np.deg2rad(1) / 1e6
DEG_PER_MYR_TO_RAD_PER_YR = 1 / 1e6

n_tde = int(tde_matrix.shape[1] / 2)
n_stations = assembly.data.n_stations
n_blocks = len(block)
n_block_constraints = assembly.data.n_block_constraints

idx = addict.Dict()
idx.start_station_row = 0
idx.end_station_row = 2 * len(station)
idx.start_block_col = 0
idx.end_block_col = len(block)
idx.start_tde_col = idx.end_block_col
idx.end_tde_col = idx.start_tde_col + n_tde
idx.meshes[0].start_tde_col = 0
idx.meshes[0].end_tde_col = 0
idx.meshes[0].start_tde_eigs_col = 0
idx.meshes[0].end_tde_eig_col = 0

# Build data vector
data_vector = np.zeros(2 * n_stations + 2 * n_tde + 3 * n_block_constraints)
data_vector[0:2 * n_stations] = celeri.interleave2(assembly.data.east_vel, assembly.data.north_vel) # add GPS data
data_vector[2 * n_stations + 2 * n_tde:2 * n_stations + 2 * n_tde + 3 * n_block_constraints] = DEG_PER_MYR_TO_RAD_PER_YR * assembly.data.block_constraints # add plate motion constraints Not sure if units are correct

# Elastic slip deficit contribution to velocities with fully locked fault segments
slip_rate_to_okada_to_velocities = copy.deepcopy(operators.okada_segment_station)
rotation_to_slip_rate = copy.deepcopy(operators.slip_rate_segment_block)

# Delete rows associated with vertical velocities
slip_rate_to_okada_to_velocities = np.delete(slip_rate_to_okada_to_velocities, np.arange(2, slip_rate_to_okada_to_velocities.shape[0], 3), axis=0)
rotation_to_slip_rate_to_okada_to_velocities = slip_rate_to_okada_to_velocities @ rotation_to_slip_rate
rotation_to_velocities = np.delete(operators.block_rotation, np.arange(2, operators.block_rotation.shape[0], 3), axis=0)

# Print sizes of matrices that we'll use to build operator
print("data_vector :" + str(data_vector.shape))
print("operators.block_motion_constraints.shape :" + str(operators.block_motion_constraints.shape))
print("rotation_to_velocities.shape :" + str(rotation_to_velocities.shape))
print("rotation_to_slip_rate_to_okada_to_velocities.shape :" + str(rotation_to_slip_rate_to_okada_to_velocities.shape))
print("tde_matrix.shape: " + str(tde_matrix.shape))
print("smoothing_matrix.shape" + str(smoothing_matrix.shape))
print("tde_and_smoothing_matrix.shape" + str(tde_and_smoothing_matrix.shape))
print("n_tde: " + str(n_tde))

# Build linear operator
operator = np.zeros((2 * assembly.data.n_stations # start rows
                     + 2 * n_tde
                     + 3 * assembly.data.n_block_constraints,
                       3 * len(block) # start columns
                     + 2 * n_tde))
print("operator.shape: " + str(operator.shape))
print("Finished printing")

# Insert block rotation and fully coupled segments
operator[0:2 * n_stations, 0:3 * n_blocks] = rotation_to_velocities - rotation_to_slip_rate_to_okada_to_velocities

# Insert block motion constraints
operator[2 * n_stations + 2 * n_tde:2 * n_stations + 2 * n_tde + 3 * n_block_constraints, 0:3 * n_blocks] = operators.block_motion_constraints

# Insert tdes and smoothing matrix
print(operator.shape)
print(2 * n_stations + 2 * n_tde)
print(3 * n_blocks + 2 * n_tde)
print(operator[0:2 * n_stations + 2 * n_tde, 3 * n_blocks : 3 * n_blocks + 2 * n_tde].shape)
print(tde_and_smoothing_matrix.shape)

operator[0:2 * n_stations + 2 * n_tde, 3 * n_blocks : 3 * n_blocks + 2 * n_tde] = tde_and_smoothing_matrix

estimation3 = addict.Dict()
state_vector3 = np.linalg.inv(operator.T @ operator) @ operator.T @ data_vector 
estimation3.predictions = operator @ state_vector3
vel = estimation3.predictions[0:2 * n_stations]
estimation3.east_vel = vel[0::2]
estimation3.north_vel = vel[1::2]

# Calculate mean squared residual velocity
east_vel_residual = estimation3.east_vel - station.east_vel
north_vel_residual = estimation3.north_vel - station.north_vel
residual = np.concatenate((east_vel_residual, north_vel_residual))
print(np.sum(residual**2) / len(station))

data_vector :(7057,)
operators.block_motion_constraints.shape :(3, 93)
rotation_to_velocities.shape :(3372, 93)
rotation_to_slip_rate_to_okada_to_velocities.shape :(3372, 93)
tde_matrix.shape: (3372, 3682)
smoothing_matrix.shape(3682, 3682)
tde_and_smoothing_matrix.shape(7054, 3682)
n_tde: 1841
operator.shape: (7057, 3775)
Finished printing
(7057, 3775)
7054
3775
(7054, 3682)
(7054, 3682)
3.7155464167624968


# Plot observed and estimated velocities with block rotations, fully locked segments, and TDEs 

In [33]:
# Extract TDE slip rates from state vector
tde_rates = state_vector3[3 * n_blocks : 3 * n_blocks + 2 * n_tde]
tde_strike_slip_rates = tde_rates[0::2]
tde_dip_slip_rates = tde_rates[1::2]

# Extract segment slip rates from state vector
slip_rates = rotation_to_slip_rate @ state_vector3[0:3 * n_blocks]
strike_slip_rates = slip_rates[0::3]
dip_slip_rates = slip_rates[1::3]
tensile_slip_rates = slip_rates[2::3]

# TODO: Calculate slip rate uncertainties

# Calculate rotation only velocities
rotation_velocities = rotation_to_velocities @ state_vector[0:3*n_blocks]

# Calculate fully locked segment velocities
elastic_segment_velocities = rotation_to_slip_rate_to_okada_to_velocities @ state_vector3[0:3*n_blocks]

# Calculate TDE velocities
tde_velocities = tde_matrix @ state_vector3[3*n_blocks:]

# Plot a lot of quantitites for a quick overview
lon_min=230
lon_max=250
lat_min=30
lat_max=52
quiver_scale=1e2
plt.figure(figsize=(12, 14))
ax1 = plt.subplot(3, 3, 1)
plt.title("observed velocities")
for i in range(len(segment)):
    plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-k", linewidth=0.5)
plt.quiver(station.lon, station.lat, station.east_vel, station.north_vel, scale=quiver_scale, scale_units="inches", color="red")
plt.xlim([lon_min, lon_max])
plt.ylim([lat_min, lat_max])
plt.gca().set_aspect("equal", adjustable="box")

plt.subplot(3, 3, 2, sharex=ax1, sharey=ax1)
plt.title("model velocities")
for i in range(len(segment)):
    plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-k", linewidth=0.5)
plt.quiver(station.lon, station.lat, estimation3.east_vel, estimation3.north_vel, scale=quiver_scale, scale_units="inches", color="blue")
plt.xlim([lon_min, lon_max])
plt.ylim([lat_min, lat_max])
plt.gca().set_aspect("equal", adjustable="box")

plt.subplot(3, 3, 3, sharex=ax1, sharey=ax1)
plt.title("residual velocities")
for i in range(len(segment)):
    plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-k", linewidth=0.5)
plt.quiver(station.lon, station.lat, station.east_vel - estimation3.east_vel, station.north_vel - estimation3.north_vel, scale=quiver_scale, scale_units="inches", color="green")
plt.xlim([lon_min, lon_max])
plt.ylim([lat_min, lat_max])
plt.gca().set_aspect("equal", adjustable="box")

plt.subplot(3, 3, 4, sharex=ax1, sharey=ax1)
plt.title("rotation velocities")
for i in range(len(segment)):
    plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-k", linewidth=0.5)
plt.quiver(station.lon, station.lat, rotation_velocities[0::2], rotation_velocities[1::2], scale=quiver_scale, scale_units="inches", color="orange")
plt.xlim([lon_min, lon_max])
plt.ylim([lat_min, lat_max])
plt.gca().set_aspect("equal", adjustable="box")

plt.subplot(3, 3, 5, sharex=ax1, sharey=ax1)
plt.title("elastic segment velocities")
for i in range(len(segment)):
    plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-k", linewidth=0.5)
plt.quiver(station.lon, station.lat, elastic_segment_velocities[0::2], elastic_segment_velocities[1::2], scale=quiver_scale, scale_units="inches", color="magenta")
plt.xlim([lon_min, lon_max])
plt.ylim([lat_min, lat_max])
plt.gca().set_aspect("equal", adjustable="box")

plt.subplot(3, 3, 6, sharex=ax1, sharey=ax1)
plt.title("elastic tde velocities")
for i in range(len(segment)):
    plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-k", linewidth=0.5)
plt.quiver(station.lon, station.lat, tde_velocities[0::2], tde_velocities[1::2], scale=quiver_scale, scale_units="inches", color="black")
plt.xlim([lon_min, lon_max])
plt.ylim([lat_min, lat_max])
plt.gca().set_aspect("equal", adjustable="box")

plt.subplot(3, 3, 7, sharex=ax1, sharey=ax1)
plt.title("segment strike-slip rates")
for i in range(len(segment)):
    plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-k", linewidth=0.5)
    plt.text(segment.mid_lon_plate_carree[i], segment.mid_lat_plate_carree[i], f"{strike_slip_rates[i]:.1f}", color="red", clip_on=True, horizontalalignment="center", verticalalignment="center")
plt.xlim([lon_min, lon_max])
plt.ylim([lat_min, lat_max])
plt.gca().set_aspect("equal", adjustable="box")

plt.subplot(3, 3, 8, sharex=ax1, sharey=ax1)
plt.title("segment dip-slip rates")
for i in range(len(segment)):
    plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-k", linewidth=0.5)
    plt.text(segment.mid_lon_plate_carree[i], segment.mid_lat_plate_carree[i], f"{dip_slip_rates[i]:.1f}", color="blue", clip_on=True, horizontalalignment="center", verticalalignment="center")
plt.xlim([lon_min, lon_max])
plt.ylim([lat_min, lat_max])
plt.gca().set_aspect("equal", adjustable="box")

plt.subplot(3, 3, 9, sharex=ax1, sharey=ax1)
plt.title("segment tensile-slip rates")
for i in range(len(segment)):
    plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-k", linewidth=0.5)
    plt.text(segment.mid_lon_plate_carree[i], segment.mid_lat_plate_carree[i], f"{tensile_slip_rates[i]:.1f}", color="green", clip_on=True, horizontalalignment="center", verticalalignment="center")
plt.xlim([lon_min, lon_max])
plt.ylim([lat_min, lat_max])
plt.gca().set_aspect("equal", adjustable="box")


# plt.subplot(2, 3, 6)
# plt.title("mesh slip rates")
# for i in range(len(segment)):
#     plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-k", linewidth=0.5)
# plt.scatter(meshes[0].centroids[:, 0], meshes[0].centroids[:, 1], s=5, c=tde_dip_slip_rates, alpha=0.5)
# plt.colorbar()
# plt.quiver(station.lon, station.lat, station.east_vel, station.north_vel, scale=quiver_scale, scale_units="inches", color="g")
# plt.quiver(station.lon, station.lat, estimation3.east_vel, estimation3.north_vel, scale=quiver_scale, scale_units="inches", color="r")
# plt.xlim([lon_min, lon_max])
# plt.ylim([lat_min, lat_max])
# plt.gca().set_aspect("equal", adjustable="box")
# plt.suptitle("block rotations + elastic segments + TDEs")
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
# Compare residuals
east_vel_residual = estimation.east_vel - station.east_vel
north_vel_residual = estimation.north_vel - station.north_vel
residual = np.concatenate((east_vel_residual, north_vel_residual))

east_vel_residual2 = estimation2.east_vel - station.east_vel
north_vel_residual2 = estimation2.north_vel - station.north_vel
residual2 = np.concatenate((east_vel_residual2, north_vel_residual2))

east_vel_residual3 = estimation3.east_vel - station.east_vel
north_vel_residual3 = estimation3.north_vel - station.north_vel
residual3 = np.concatenate((east_vel_residual3, north_vel_residual3))

mean_squared_residual1 = np.sum(residual**2) / len(station)
mean_squared_residual2 = np.sum(residual2**2) / len(station)
mean_squared_residual3 = np.sum(residual3**2) / len(station)

n_bins = 100
plt.figure()
n, bins, _ = plt.hist(residual, bins=np.linspace(-10, 10, n_bins), density=False, histtype='step', label=f"block rotations : {mean_squared_residual1:.2f}")
n, bins, _ = plt.hist(residual2, bins=np.linspace(-10, 10, n_bins), density=False, histtype='step', label=f"block rotations + elastic segments : {mean_squared_residual2:.2f}")
n, bins, _ = plt.hist(residual3, bins=np.linspace(-10, 10, n_bins), density=False, histtype='step', label=f"block rotations + elastic segments + tdes : {mean_squared_residual3:.2f}")
plt.legend()
plt.xlabel("residual (mm/yr)")
plt.ylabel("N")
plt.xlim(-10, 10)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
segment.keys()

Index(['name', 'lon1', 'lat1', 'lon2', 'lat2', 'dip', 'res', 'other3',
       'other6', 'other7', 'other8', 'other9', 'other10', 'other11', 'other12',
       'locking_depth', 'locking_depth_sig', 'locking_depth_flag', 'dip_sig',
       'dip_flag', 'ss_rate', 'ss_rate_sig', 'ss_rate_flag', 'ds_rate',
       'ds_rate_sig', 'ds_rate_flag', 'ts_rate', 'ts_rate_sig', 'ts_rate_flag',
       'burial_depth', 'burial_depth_sig', 'burial_depth_flag',
       'resolution_override', 'resolution_other', 'patch_file_name',
       'patch_flag', 'patch_slip_file', 'patch_slip_flag', 'length', 'x1',
       'y1', 'z1', 'x2', 'y2', 'z2', 'mid_lon_plate_carree',
       'mid_lat_plate_carree', 'mid_lon', 'mid_lat', 'mid_x', 'mid_y', 'mid_z',
       'centroid_x', 'centroid_y', 'centroid_z', 'centroid_lon',
       'centroid_lat', 'west_labels', 'east_labels'],
      dtype='object')

In [14]:
f'{slip_rates[0]:.2f}'

'-0.29'

In [15]:
np.deg2rad(1)

0.017453292519943295

In [29]:
state_vector

array([ 1.24789334e-07,  6.92353654e-07, -9.63634324e-07,  8.29174986e-07,
        1.72787704e-06, -2.23852899e-06, -2.16023878e-06,  5.30327165e-06,
       -7.52549957e-06, -9.10991565e-07,  8.24837901e-06, -1.01437522e-05,
       -2.10461151e-07,  2.60307424e-06, -3.21085490e-06, -4.09594707e-06,
       -3.75160356e-06,  1.81280705e-06,  5.85928188e-07, -2.41217202e-07,
        1.02499999e-06,  8.94736864e-07,  2.57081230e-06, -2.74180016e-06,
        2.95216092e-06,  5.72399429e-06, -6.55079171e-06,  4.96697956e-06,
        9.90628451e-06, -1.03685639e-05,  2.88908824e-06,  5.71370878e-06,
       -6.42910094e-06,  3.95614856e-06,  7.86362239e-06, -8.66097198e-06,
       -1.33654244e-06, -4.43394769e-07, -6.80660304e-07, -1.82565493e-06,
        7.58653859e-08, -1.68734285e-06,  8.28663093e-07,  3.59280184e-06,
       -4.91272721e-06, -7.46624073e-06, -6.28199992e-06,  1.67480795e-06,
        6.10727764e-06,  1.31274315e-05, -1.35512784e-05, -1.66446618e-05,
       -2.42748035e-05,  